In [4]:
# A script download digital painting time lapse videos from YouTube (there are a few Vimeo videos that will need to be 
# downloaded manually). Once the video files are saved to disk, this script will extract the specified
# frames, crop them to the painting, and save them in a pkl file.

# Run these if you do not have the dependencies yet
#!pip install pytube3
#!pip install sk-video

import os
from pytube import YouTube
import pickle
import skvideo.io as skvio

vid_info_dir = ""  # location where you downloaded the digital painting .pkl files
out_vids_dir = ""  # location to download videos to
out_vid_info_dir = ""  # location to save .pkls (with frame information -- will be LARGE files!) to

pkl_filenames = [f for f in os.listdir(vid_info_dir) if f.endswith('.pkl')]

# Download each video from YouTube
did_not_download = []
for pkl_filename in pkl_filenames:
    vid_name = pkl_filename.strip('.pkl')
    
    # the first hyphen splits the video owner name and video id
    vid_id = vid_name[vid_name.index('_') + 1:]
    
    vid_url = f'https://www.youtube.com/watch?v={vid_id}'
    print('Attempting to download {}'.format(vid_url))


    vid_file = os.path.join(out_vids_dir, '{}.mp4'.format(vid_name))
    vid_frames_dir = os.path.join(out_vid_info_dir, vid_name)
        
    yt = YouTube(vid_url)

    # if we have the video already, don't bother tryinkg to download it
    if not os.path.isfile(vid_file):
        for s in yt.streams.all():
            if s is None or s.resolution is None or s.mime_type is None:
                continue
            if '360' in s.resolution and "mp4" in s.mime_type:
                try:
                    print('Downloading stream {} to file {}'.format(s, vid_file))
                    # download the video in 360p
                    s.download(out_vids_dir, filename=vid_name)
                    break
                except:
                    print('Could not download {}, trying another...'.format(s))

        if not os.path.isfile(vid_file):
            print('Could not download {}, skipping! Try looking on Vimeo manually for the same ID.'.format(vid_name))
            did_not_download.append(vid_file)
            continue

    if not os.path.isdir(vid_frames_dir):
        os.mkdir(vid_frames_dir)

    # load the information we need to extract the right frames
    with open(os.path.join(vid_info_dir, pkl_filename), 'rb') as pf:
        vid_info = pickle.load(pf) 
    if 'frames' not in vid_info.keys():
        vid_info['frames'] = [None] * len(vid_info['framenums'])
    # now open the video file we just downloaded, and extract the frames from it
    reader = skvio.FFmpegReader(vid_file)
    
    for i, frame in enumerate(reader.nextFrame()):
        if i in vid_info['framenums']:
            frame = frame[..., [2, 1, 0]]  # make it into BGR format for opencv
            
            # crop it according to the saved croprect
            frame = frame[int(vid_info['crop_start_xy'][1]):int(vid_info['crop_end_xy'][1]), int(vid_info['crop_start_xy'][0]):int(vid_info['crop_end_xy'][0])]
            frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
            vid_info['frames'][vid_info['framenums'].index(i)] = frame
            
    with open(os.path.join(out_vid_info_dir, pkl_filename), 'rb') as pf:
        pickle.dump(vid_info, pf, protocol=4)  # protocol 4 allows for larger file sizes
        
print('Did not download the following files: {}'.format(did_not_download))


Attempting to download https://www.youtube.com/watch?v=sm-L8G_-x4I


AssertionError: Cannot find installation of real FFmpeg (which comes with ffprobe).